# Generate multiple data sets to vary criteria values

Afin de générer plusieurs jeux de données test qui vont influés sur les valeurs des critères, il faut d'abord identifier ce que les scores des facteurs relatent réellement sur notre base de données et ainsi, identifier ce qui peut déteriorer la valeur de chaque critère : 
- MixedColumnScore : valeurs aberrantes
- ColumnsScore : teste si il y a une majorité (string ou int) dans ma colonne (10% -> 0 ou 1)
- diversityScore : insérer du bruit dans nos données pour qu'elles soient moins stables (+ limiter la corrélation avec la valeur target finale)
- cellsCompletnessScore : insérer des valeurs nulles dans mon dataset => Correspond à la proportion de NULL, il suffit donc d'en injecter
- consistentRepresentationScore : détecte l'utilisation de synonymes pour décrire une seule et même chose (purement textuel)

In [1]:
import pandas as pd
import numpy as np
import os
import random

In [2]:
from MixedColumnDetector import res
from Z_Score import printresult
from CompletnessPart1 import completeness_score
from DIversificationScore import global_dataset_score

outliersZcoreScore = res('outliers_heart.csv') 
outliersColumnsScore = printresult('outliers_heart.csv') 
diversityScore = global_dataset_score('outliers_heart.csv')
cellsCompletnessScore = completeness_score('outliers_heart.csv')

[1.28475334 0.77234554 1.29941848 1.28894338 0.77234554 0.77234554
 1.29208591 0.77234554 1.29858047 0.77234554 1.29208591 0.77234554
 0.77234554 0.77234554 1.29459993 0.77234554 0.77234554 0.77234554
 0.77234554 0.77234554 1.29522844 1.29858047 0.77234554 0.77234554
 0.77234554 0.77234554 0.77234554 1.29732346 0.77234554 0.77234554
 0.77234554 1.29732346 1.28475334 1.29313342 0.77234554 0.77234554
 0.77234554 1.28894338 0.77234554 0.77234554 1.2910384  0.77234554
 0.77234554 0.77234554 1.29313342 1.2910384  0.77234554 0.77234554
 1.29313342 1.2914574  0.77234554 0.77234554 1.2910384  1.29397143
 1.29522844 0.77234554 0.77234554 0.77234554 1.29941848 1.28894338
 1.29334292 1.28517235 0.77234554 0.77234554 1.29627595 1.29313342
 0.77234554 0.77234554 0.77234554 1.2910384  0.77234554 1.28894338
 0.77234554 0.77234554 0.77234554 1.2910384  0.77234554 0.77234554
 0.77234554 1.30151349 1.30046599 1.2910384  0.77234554 1.29522844
 0.77234554 1.29941848 1.29732346 0.77234554 0.77234554 0.7723

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs - f_exp)**2 / f

In [3]:
print(outliersZcoreScore)
print(outliersColumnsScore)
print(diversityScore)
print(cellsCompletnessScore)

1
0.6263616557734204
0.617014717956334
1.0


## Création de nouvelles BD

In [4]:
# Recharger le dataset d'origine
df = pd.read_csv("heart.csv")

# Créer un dossier de sortie
output_dir = "output_corrupt_data"
os.makedirs(output_dir, exist_ok=True)

In [8]:
# 1. Dégradation pour MixedColumnScore (valeurs aberrantes)
df_mixed = df.copy()
for col in df_mixed.select_dtypes(include=[np.number]).columns:
    # Injecter des valeurs aberrantes dans 5% des lignes
    indices = df_mixed.sample(frac=0.40).index
    df_mixed.loc[indices, col] *= np.random.randint(10, 100)
df_mixed.to_csv(f"{output_dir}/heart_mixed_column_score.csv", index=False)

file_name = "heart_mixed_column_score"
print("-- MixedColumnScore : valeurs aberrantes")
print("ancienne valeur : ", outliersZcoreScore)
print("nouvelle valeur :",res(f"{output_dir}/{file_name}.csv"))
print()
print("-- diversityScore : stabilité des données")
print("ancienne valeur : ", diversityScore)
print("nouvelle valeur :",global_dataset_score(f"{output_dir}/{file_name}.csv"))
print()
print("-- cellsCompletnessScore : valeurs nulles")
print("ancienne valeur : ", cellsCompletnessScore)
print("nouvelle valeur :",completeness_score(f"{output_dir}/{file_name}.csv"))


-- MixedColumnScore : valeurs aberrantes
ancienne valeur :  1
nouvelle valeur : 1

-- diversityScore : stabilité des données
ancienne valeur :  0.617014717956334
nouvelle valeur : 0.6706330765947145

-- cellsCompletnessScore : valeurs nulles
ancienne valeur :  1.0
nouvelle valeur : 1.0


In [ ]:
# 2. Dégradation pour ColumnsScore (proportion de int/str)
df_mixted_column = df.copy()


In [ ]:
# 3. Dégradation pour diversityScore (instabilité)
df_diverse = df.copy()

# Injecter du bruit aléatoire dans des colonnes catégorielles et numériques
for col in df_diverse.columns:
    if df_diverse[col].dtype == "object":
        df_diverse.loc[df_diverse.sample(frac=0.1).index, col] = "??"
    elif df_diverse[col].dtype in [np.int64, np.float64]:
        # Ajouter un bruit aléatoire sur 10% des lignes
        indices = df_diverse.sample(frac=0.1).index
        df_diverse.loc[indices, col] += np.random.normal(0, 10, size=len(indices))
df_diverse.to_csv(f"{output_dir}/heart_diversity_score.csv", index=False)

file_name = "heart_diversity_score"
print("-- MixedColumnScore : valeurs aberrantes")
print("ancienne valeur : ", outliersZcoreScore)
print("nouvelle valeur :",res(f"{output_dir}/{file_name}.csv"))
print()
print("-- diversityScore : stabilité des données")
print("ancienne valeur : ", diversityScore)
print("nouvelle valeur :",global_dataset_score(f"{output_dir}/{file_name}.csv"))
print()
print("-- cellsCompletnessScore : valeurs nulles")
print("ancienne valeur : ", cellsCompletnessScore)
print("nouvelle valeur :",completeness_score(f"{output_dir}/{file_name}.csv"))

/var/folders/w0/8c1n56050zj35dsmfbf6htkr0000gn/T/ipykernel_4218/1855233054.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[22.45987856 67.60363573 58.93914314 68.49011022 24.55383837 74.88085122
 44.62529091 40.03997317 34.83291658 43.08730773 71.62840431 64.66215145
 57.41965572 17.64399477 69.05897697 82.69206377 68.82427359 57.14763375
 76.33732754 50.29331169 32.03734831 72.88885209 68.16305297 78.39367915
 45.10470494 57.02999514 53.95459768 45.91513188 43.3911666  45.90548317
 63.29029836 39.85627153 68.52482501 21.5745229  60.86314308 70.76231501
 41.85574275 54.37471069 36.88229224 40.43207119 56.36369297 32.6049482
 61.75513822 55.5473663  66.81596228 36.09002351 58.78069311 47.57484113
 42.99185462 34.72746902 59.86080442 70.35266711 29.43098228 59.75143203
 62.53036151 68.95661003 76.93966476 69.44351793 53.61547442 46.10336394
 45.84790667 54.6785678  59.58408498 71.62277488 68.77878405

-- MixedColumnScore : valeurs aberrantes
ancienne valeur :  1
nouvelle valeur : 1

-- diversityScore : stabilité des données
ancienne valeur :  0.617014717956334
nouvelle valeur : 0.3768370368578212

-- cellsCompletnessScore : valeurs nulles
ancienne valeur :  1.0
nouvelle valeur : 1.0


In [ ]:
# 4. Dégradation pour cellsCompletnessScore (valeurs nulles)
df_nulls = df.copy()
# Pour chaque colonne, rendre 50% des cellules nulles aléatoirement
for col in df_nulls.columns:
    null_indices = df_nulls.sample(frac=0.5).index
    df_nulls.loc[null_indices, col] = np.nan

df_nulls.to_csv(f"{output_dir}/heart_cells_completeness_score.csv", index=False)

file_name = "heart_cells_completeness_score"
print("-- MixedColumnScore : valeurs aberrantes")
print("ancienne valeur : ", outliersZcoreScore)
print("nouvelle valeur :",res(f"{output_dir}/{file_name}.csv"))
print()
print("-- diversityScore : stabilité des données")
print("ancienne valeur : ", diversityScore)
print("nouvelle valeur :",global_dataset_score(f"{output_dir}/{file_name}.csv"))
print()
print("-- cellsCompletnessScore : valeurs nulles")
print("ancienne valeur : ", cellsCompletnessScore)
print("nouvelle valeur :",completeness_score(f"{output_dir}/{file_name}.csv"))


-- MixedColumnScore : valeurs aberrantes
ancienne valeur :  1
nouvelle valeur : 0

-- diversityScore : stabilité des données
ancienne valeur :  0.617014717956334


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pingouin/correlation.py:59: RuntimeWarning: invalid value encountered in sqrt
  tval = r * np.sqrt(dof / (1 - r**2))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pingouin/effsize.py:153: RuntimeWarning: invalid value encountered in sqrt
  se = 1 / np.sqrt(nx - 3)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pingouin/correlation.py:59: RuntimeWarning: invalid value encountered in sqrt
  tval = r * np.sqrt(dof / (1 - r**2))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pingouin/effsize.py:153: RuntimeWarning: invalid value encountered in sqrt
  se = 1 / np.sqrt(nx - 3)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pingouin/correlation.py:59: RuntimeWarning: invalid value encountered in sqrt
  tval = r * np.sqrt(dof / (1 - r**2))
/Library/Frameworks/Python.framework/Versions

nouvelle valeur : 0.3531030455976833

-- cellsCompletnessScore : valeurs nulles
ancienne valeur :  1.0
nouvelle valeur : 0.5
